In [1]:
import tensorflow as tf 

2024-01-31 18:33:14.981449: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 18:33:14.981610: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 18:33:15.016295: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-31 18:33:15.101857: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-31 18:33:17.002623: W tensorflow/compiler/tf2

In [3]:
model = tf.keras.models.load_model('toxicity_model(12).h5')

2024-01-31 18:33:58.218701: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [4]:

import os 
import matplotlib.pyplot as plt 
import tensorflow as tf
import pandas as pd 
import numpy as np

In [18]:
df = pd.read_csv(os.path.join('ToxicCommandDataset','train','train.csv'))

In [19]:
X = df["comment_text"]
y = df.drop(columns=["comment_text","id"], axis=1)

In [7]:
from tensorflow.keras.layers import  TextVectorization

In [8]:
y = y.values

In [9]:
#initalise how many wors are stored inside that vocab
MAX_WORDS = 200000  #number of words in the vocab

In [10]:
vectorizer = TextVectorization(max_tokens=MAX_WORDS , #number of words in the vocab
    output_sequence_length = 1800 , #it the length of the outsequense  
    output_mode = 'int'#the word number is integear
    )

In [11]:
vectorizer.adapt(X.values)

In [12]:
input_text = vectorizer("What the Fuck!")

In [16]:
pred =model.predict(np.expand_dims(input_text,0))

1/1 [==============================] - 0s 153ms/step


In [20]:
y.columns

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [17]:
(pred > 0.5).astype(int)

array([[1, 0, 1, 0, 1, 0]])

In [21]:
import tensorflow as tf
import gradio as gr

/home/saaho/Mechine_learnig/DeepLearning/MODELS/ToxicCommandClassify/ToxicCommand/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)
    
    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
    
    return text

In [24]:
interface = gr.Interface(fn=score_comment, 
                         inputs=gr.Textbox(lines=2, placeholder='Comment to score'),
                        outputs='text')

In [ ]:
interface.launch(share=True)
